In [210]:
%pip install selenium
%pip install bs4


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [211]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [212]:
driver = webdriver.Firefox()
driver.maximize_window()

In [213]:
driver.get("https://www.vino.com/en/s/?nav=true")

In [214]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [215]:
urls = [
    "https://www.vino.com/en/vino/rosso",
    "https://www.vino.com/en/vino/bianco",
    "https://www.vino.com/en/vino/spumante",
    "https://www.vino.com/en/vino/champagne",
    "https://www.vino.com/en/vino/rosato",
    "https://www.vino.com/en/vino/fortificato",
    "https://www.vino.com/en/vino/passito",
    "https://www.vino.com/en/altro/olio-extravergine",
    "https://www.vino.com/en/altro/aceti-balsamici",
    "https://www.vino.com/en/altro/accessori",
    "https://www.vino.com/en/altro/birra",
    "https://www.vino.com/en/altro/distillato",
    "https://www.vino.com/en/vino/finewine",
    "https://www.vino.com/en/s/?nav=true&ultimaScelta=abbinamenti&categoria=mixology",
    "https://www.vino.com/en/s/?nav=true&ultimaScelta=abbinamenti&categoria=vini",

    "https://www.vino.com/en/distillati/brandy",
    "https://www.vino.com/en/distillati/acquavite",
    "https://www.vino.com/en/distillati/gin",
    "https://www.vino.com/en/distillati/grappa",
    "https://www.vino.com/en/distillati/liquore",
    "https://www.vino.com/en/distillati/rum",
    "https://www.vino.com/en/distillati/tequila",
    "https://www.vino.com/en/distillati/vermouth",
    "https://www.vino.com/en/distillati/vodka",
    "https://www.vino.com/en/distillati/whisky",
]

In [216]:
def per_vine_url(driver):
    print("per vine")
    urls = []
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    rows = soup.find_all('div', class_="col-xs-3 search-result-bottle-grid-col-left text-center")
    for row in rows:
        link = f"https://www.vino.com/en/{row.select_one('a')['href']}"
        urls.append(link)
    return urls


# vine_urls = per_vine_url(driver=driver)

In [217]:
repos = list()
def data_collect(content):   
    
    big_data = {'Image': '', 'Type of wine': '', 'Body': '', 'Origin': '', 'Ideal with': '', 'Aroma': '',
                'Full name': '', 'Appellation': '', 'Vintage': '', 'Dosage': '', 'Country': '', 'Region': '',
                'Grape': '', 'Ageing': '', 'Format': '', 'Alcohol content': '', 'Temperature': '', 'Sulfites': '',
                'Average Rating': '', 'Price': '', 'Description': ''}
    
    result_question = content.find_all("div", class_="col-xs-4 nome")
    result_answer = content.find_all("div", class_="col-xs-8 descrizione")
    values_question = [desc.text.strip() for desc in result_question]
    values_answer = [desc.text.strip() for desc in result_answer]

    try:
        big_data['Image'] = content.find("link", itemprop="image")["content"]
    except:
        big_data['Image'] = np.nan
    try:
        big_data['Type of wine'] = content.find("div", class_="col-xs-12 m-t-10").text.strip("\n").split("\n")[1]
    except:
        big_data['Type of wine'] = np.nan
    try:
        big_data['Body'] = content.find("div", class_="col-xs-12 m-t-10").text.strip("\n").split("\n")[13]
    except:
        big_data['Body'] = np.nan
    try:
        big_data['Origin'] = content.find("div", class_="col-xs-12 m-t-10").text.strip("\n").split("\n")[28]
    except:
        big_data['Origin'] = np.nan
    try:
        big_data['Ideal with'] = content.find("div", class_="col-xs-12 m-t-10").text.strip("\n").split("\n")[37]
    except:
        big_data['Ideal with'] = np.nan
    try:
        big_data['Aroma'] = content.find("div", class_="col-xs-12 m-t-10").text.strip("\n").split("\n")[19]
    except:
        big_data['Aroma'] = np.nan

    try:
        if values_question[0] == 'full name':
            big_data['Full name'] = values_answer[0]
    except:
        big_data['Full name'] = np.nan
    try:
        if values_question[1] == 'appellation':
            big_data['Appellation'] = values_answer[1]
    except:
        big_data['Appellation'] = np.nan
    try:
        if values_question[2] == 'vintage':
            big_data['Vintage'] = values_answer[2]
    except:
        big_data['Vintage'] = np.nan
    try:
        if values_question[3] == 'dosage':
            big_data['Dosage'] = values_answer[3]
    except:
        big_data['Dosage'] = np.nan
    try:
        if values_question[4] == 'country':
            big_data['Country'] = values_answer[4]
    except:
        big_data['Country'] = np.nan
    try:
        if values_question[5] == 'region':
            big_data['Region'] = values_answer[5]
    except:
        big_data['Region'] = np.nan
    try:
        if values_question[6] == 'grape':
            big_data['Grape'] = values_answer[6]
    except:
        big_data['Grape'] = np.nan
    try:
        if values_question[7] == 'ageing':
            big_data['Ageing'] = values_answer[7]
    except:
        big_data['Ageing'] = np.nan
    try:
        if values_question[8] == 'format':
            big_data['Format'] = values_answer[8]
    except:
        big_data['Format'] = np.nan
    try:
        if values_question[10] == 'alcohol content':
            big_data['Alcohol content'] = values_answer[10]
    except:
        big_data['Alcohol content'] = np.nan
    try:
        if values_question[11] == 'temperature':
            big_data['Temperature'] = values_answer[11]
    except:
        big_data['Temperature'] = np.nan
    try:
        if values_question[12] == 'Sulfites':
            big_data['Sulfites'] = values_answer[12]
    except:
        big_data['Sulfites'] = np.nan
        
    try:    
        big_data['Average Rating'] = content.find("span", id="rating-average").text,
    except:
        big_data['Average Rating'] = np.nan
    try:
        big_data['Price'] = content.find("span", class_='prezzo watc-sale-price').text,
    except:
        big_data['Price'] = np.nan
    try:
        big_data['Description'] = content.find("div", itemprop="description").text
    except:
        big_data['Description'] = np.nan
    repos.append(big_data)

In [218]:
def enter_each_url(urls):
    for url in urls:
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        data_collect(soup)

# enter_each_url(vine_urls)

In [219]:
for url in urls:
    print(url)
    exp = 0
    driver.get(url)
    for i in range(50):
        if i < 6:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            driver.execute_script("window.scrollBy(0, -500);")
            time.sleep(1.3)
        elif exp == 2:
            continue
        else:
            try:
                button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'btn.btn-secondary.btn-sm'))
                )
                button.click()
                time.sleep(2)
            except:
                exp += 1
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                driver.execute_script("window.scrollBy(0, -500);")
                time.sleep(1)
                continue

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            driver.execute_script("window.scrollBy(0, -500);")
                
    vine_urls = per_vine_url(driver=driver)
    enter_each_url(vine_urls)

https://www.vino.com/en/vino/rosso
per vine
https://www.vino.com/en/dettaglio/brunello-di-montalcino-docg-fattoria-dei-barbi-2018.html
https://www.vino.com/en/dettaglio/sicilia-doc-nero-avola-inessa-2022.html
https://www.vino.com/en/dettaglio/terre-siciliane-syrah-igt-lu-meli-li-ciuri-2021.html
https://www.vino.com/en/dettaglio/terre-siciliane-syrah-igt-li-ciuri-2021.html
https://www.vino.com/en/dettaglio/barbera-d-alba-doc-sabaudo-2021.html
https://www.vino.com/en/dettaglio/bolgheri-rosso-doc-campo-al-faro-2021.html
https://www.vino.com/en/dettaglio/chianti-classico-docg-borgo-san-frediano-2021.html
https://www.vino.com/en/dettaglio/amarone-della-valpolicella-docg-niviana-2018.html
https://www.vino.com/en/dettaglio/barolo-docg-sabaudo-2019.html
https://www.vino.com/en/dettaglio/langhe-nebbiolo-doc-sabaudo-2022.html
https://www.vino.com/en/dettaglio/chianti-classico-riserva-docg-borgo-san-frediano-2019.html
https://www.vino.com/en/dettaglio/amarone-classico-della-valpolicella-docg-cost

In [220]:
df = pd.DataFrame.from_dict(repos)

In [221]:
df.to_csv('aa.csv', index=False)

In [222]:
df.shape

(2790, 21)